In [2]:
from selenium import webdriver
import time

# test code

In [2]:
import time

from selenium import webdriver


def find_urls():

    driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
    driver.implicitly_wait(3)
    driver.get('https://www.dabangapp.com/search#/map')
    driver.find_element_by_name('keyword').send_keys('성수동1가')
    driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]').click()
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[1]/a[1]').click()

    time.sleep(2)
    url_all_list =[]
    for i in range(0, 3):
        time.sleep(2)
        seongsoo_1dong_detail_list = driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")
        for data in seongsoo_1dong_detail_list:
            url_all_list.append(data.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
    print(len(url_all_list))

    driver.find_element_by_name('keyword').send_keys('성수동2가')
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]').click()
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[1]/a[1]').click()
    time.sleep(2)
    seongsoo_2dong_detail_list = driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul[2]/li/div/a")
    for data in seongsoo_2dong_detail_list:
        url_all_list.append(data.get_attribute('href'))
    for i in range(0, 2):
        time.sleep(2)
        seongsoo_2dong_detail_list = driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")
        for data in seongsoo_2dong_detail_list[2:]:
            url_all_list.append(data.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
    print(len(url_all_list))
    driver.close()
    return url_all_list



In [3]:
url_all_list = find_urls()

56
126


In [5]:
def find_apartment_urls():
    driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
    driver.get('https://www.dabangapp.com/search#/map')

    driver.find_element_by_name('keyword').send_keys('성수동1가')
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div[2]/ul[2]/li/label/span[2]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)
    
    apt_list = []
    for i in range(6):
        apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')
        for url in apart_urls:
            apt_list.append(url.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
        time.sleep(2)

    # 성수 2동 시작.
    driver.find_element_by_name('keyword').send_keys('성수동2가')
    time.sleep(2)
    
    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)
    
    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)
    
    apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')

    for url in apart_urls:
        apt_list.append(url.get_attribute('href'))
    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)


    apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')
    for url in apart_urls:
        apt_list.append(url.get_attribute('href'))


    return apt_list


In [6]:
apt_urls = find_apartment_urls()
len(apt_urls)


163

In [8]:
len(url_all_list)

126

# test code apart

In [17]:
from selenium import webdriver

driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
driver.implicitly_wait(3)

dabang_url_7 = 'https://www.dabangapp.com/search#/map?filters=%7B%22multi_room_type%22%3A%5B3%5D%2C%22selling_type%22%3A%5B0%2C1%2C2%5D%2C%22deposit_range%22%3A%5B0%2C999999%5D%2C%22price_range%22%3A%5B0%2C999999%5D%2C%22trade_range%22%3A%5B0%2C999999%5D%2C%22maintenance_cost_range%22%3A%5B0%2C999999%5D%2C%22include_maintenance_option1%22%3Atrue%2C%22room_size%22%3A%5B0%2C999999%5D%2C%22supply_space_range%22%3A%5B0%2C999999%5D%2C%22room_floor_multi%22%3A%5B1%2C2%2C3%2C4%2C5%2C6%2C7%2C-1%2C0%5D%2C%22division%22%3Afalse%2C%22duplex%22%3Afalse%2C%22room_type%22%3A%5B%5D%2C%22enter_date_range%22%3A%5B0%2C999999%5D%2C%22parking_average_range%22%3A%5B0%2C999999%5D%2C%22household_num_range%22%3A%5B0%2C999999%5D%2C%22parking%22%3Afalse%2C%22animal%22%3Afalse%2C%22short_lease%22%3Afalse%2C%22full_option%22%3Afalse%2C%22built_in%22%3Afalse%2C%22elevator%22%3Afalse%2C%22balcony%22%3Afalse%2C%22loan%22%3Afalse%2C%22safety%22%3Afalse%2C%22pano%22%3Afalse%2C%22deal_type%22%3A%5B0%2C1%5D%7D&position=%7B%22location%22%3A%5B%5B127.05104483795931%2C37.52761263948507%5D%2C%5B127.08601899892244%2C37.56175822807376%5D%5D%2C%22center%22%3A%5B127.06852792951773%2C37.5446867572873%5D%2C%22zoom%22%3A6%7D&search=%7B%22id%22%3A%2211200115%22%2C%22type%22%3A%22region%22%2C%22name%22%3A%22%EC%84%B1%EC%88%98%EB%8F%992%EA%B0%80%22%7D&tab=all'



SessionNotCreatedException: Message: session not created
from disconnected: received Inspector.detached event
  (Session info: chrome=80.0.3987.163)
  (Driver info: chromedriver=2.44.609545 (c2f88692e98ce7233d2df7c724465ecacfe74df5),platform=Mac OS X 10.14.4 x86_64)


In [156]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
driver.implicitly_wait(3)

In [157]:
bang_url_list = []

In [158]:
driver.get(dabang_url_7)
url_list = driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")
for data in url_list:
    data = data.get_attribute('href')
    bang_url_list.append(data)
try:
    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
    driver.execute_script("arguments[0].click();", button)
    url_list = driver.find_elements_by_xpath("/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")
    for data in url_list:
        data = data.get_attribute('href')
        bang_url_list.append(data)
except NoSuchElementException:
    pass


In [159]:
len(bang_url_list)

40

In [79]:
# [driver.get(url) for url in bang_url_list]

# test code start

In [80]:
from selenium.common.exceptions import NoSuchElementException

In [48]:
driver.get(bang_url_list[11])

In [49]:
import time
time.sleep(5)
# 이거 안하면 아직 페이지 로딩 안되서 에러남 버튼 아직 없음

# broker CODE

In [50]:
button = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div/ul/li[4]/button")
driver.execute_script("arguments[0].click();", button)

In [51]:
unrefined_name = driver.find_elements_by_xpath('/html/body/div[4]/div/div/div/div[1]/div[1]/h1')
name = unrefined_name[0].get_attribute('innerText')

In [52]:
name

'뚝섬포인트공인중개사사무소'

In [53]:
unrefined_address = driver.find_elements_by_xpath('/html/body/div[4]/div/div/div/div[1]/div[1]/p')
address = unrefined_address[0].get_attribute('innerText')
address

'서울특별시 성동구 성덕정길 28,1층 (성수동1가)'

In [65]:
unrefined_manager = driver.find_elements_by_xpath('/html/body/div[4]/div/div/div/div[1]/div[2]/p[1]/font')
manager = unrefined_manager[0].get_attribute('innerText')
manager

'채명수'

In [66]:
unrefined_tel = driver.find_elements_by_xpath('/html/body/div[4]/div/div/div/div[1]/div[2]/p[2]/font')
tel = unrefined_tel[0].get_attribute('innerText')
tel

'02-498-9125'

In [70]:
def get_innerText(url):
    unrefined_data = driver.find_element_by_xpath(f'{url}')
    data = unrefined_data[0].get_attribute('innerText')
    return data

In [72]:
url = '/html/body/div[4]/div/div/div/div[1]/div[2]/p[1]/font'
tel_Data = get_innerText(url)

TypeError: 'WebElement' object is not subscriptable

# find Room CODE

In [448]:
try:
    button = driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/div/div/button")
    driver.execute_script("arguments[0].click();", button)
except NoSuchElementException:
    pass    

In [449]:
unrefined_description = driver.find_elements_by_xpath("/html/body/div[1]/div/div[4]/div/div")

In [450]:
description = unrefined_description[0].get_attribute("innerText")

In [451]:
description

'인기 많은 투룸!\n★ 여자 둘이 살기 좋은 투룸 월세! (여자만 받아요~! 반지하부터 모두 여자만 살아요)\n\n\n★ 뚝섬역, 서울숲역 모두 7~8분 거리 (딱 가운데 위치)\n\n\n★ 조용한 서울숲 옆에 1층집입니당\n\n\n★ 집 근처가 인스타에서 핫한 카페, 맛집 있어요\n[ ㅊa, 제스티살룬 (30초컷), 모멘토 (30초컷), 트와블루, 서울숲 공원, 빵의정석, 쏘마이피자 등 ]\n\n\n★ 주변에 은행, 주민센터, 병원 , 성동구민체육센터 모두 10분 거리!!\n\n\n★ 출근할 때 공원 가로질러서 가서 상쾌함\n\n\n★ 퇴근할 때 카페거리고 여성안심 가로등 같은거 있어서 안무서워요\n\n\n★ 주변이 카페여서 밤에 무지무지 조용해요\n\n\n★ 전등, 보일러 등 집에 필요한 사소한거 고장나면 주인분이 다 고쳐주심!!\n\n\n★ 집 볼 수 있는 시간은 평일엔 7시반 이후!\n주말엔 시간 맞춰보면 돼요\n\n\n★ 베란다 있어서 여름에 빨래 널기 좋아요\n\n\n★ 애완동물 안돼요\n\n\n★ 옵션은 다 가져갈거여서 챙겨와야 합니당\n접기'

In [452]:
description.replace("\n", "")

'인기 많은 투룸!★ 여자 둘이 살기 좋은 투룸 월세! (여자만 받아요~! 반지하부터 모두 여자만 살아요)★ 뚝섬역, 서울숲역 모두 7~8분 거리 (딱 가운데 위치)★ 조용한 서울숲 옆에 1층집입니당★ 집 근처가 인스타에서 핫한 카페, 맛집 있어요[ ㅊa, 제스티살룬 (30초컷), 모멘토 (30초컷), 트와블루, 서울숲 공원, 빵의정석, 쏘마이피자 등 ]★ 주변에 은행, 주민센터, 병원 , 성동구민체육센터 모두 10분 거리!!★ 출근할 때 공원 가로질러서 가서 상쾌함★ 퇴근할 때 카페거리고 여성안심 가로등 같은거 있어서 안무서워요★ 주변이 카페여서 밤에 무지무지 조용해요★ 전등, 보일러 등 집에 필요한 사소한거 고장나면 주인분이 다 고쳐주심!!★ 집 볼 수 있는 시간은 평일엔 7시반 이후!주말엔 시간 맞춰보면 돼요★ 베란다 있어서 여름에 빨래 널기 좋아요★ 애완동물 안돼요★ 옵션은 다 가져갈거여서 챙겨와야 합니당접기'

# 이건 데이터 어떻게 념겨줄건지 이야기 할 때 내가 안바꾸고 거기서 바꾸는게 더 편하지 않겠냐 물어보기 -- 위에 출력한 텍스트 두개

In [453]:
unrefined_address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div/div/p')
address = unrefined_address[0].get_attribute("innerText")
# 이건 테이블 분리 안하고 그냥 여기에 넣으면 될 듯 동 단위여서 의미가 없음
# 테이블 수정함 PostAddress
address

'서울특별시 성동구 성수동1가'

In [454]:
unrefined_salesform = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/div')
salesForm = unrefined_salesform[0].get_attribute("innerText")
salesForm

'월세 2000/55\n만원'

In [455]:
salesForm = salesForm.replace('/', ' ')

In [456]:
salesForm = salesForm.replace('\n', '')

In [457]:
salesForm

'월세 2000 55만원'

In [458]:
salesForm = salesForm.split()

In [459]:
salesForm

['월세', '2000', '55만원']

In [460]:
salesType = salesForm[0]

In [461]:
salesType

'월세'

In [462]:
salesDepositChar = salesForm[1]

In [463]:
salesDepositChar

'2000'

In [464]:
if salesDepositChar.find('억'):
    print(1)
else:
    print(2)

1


In [465]:
if salesDepositChar.find('원'):
    print(salesDepositChar)
    salesDepositChar = salesDepositChar.replace('원','')
    print('바뀜')
    print(salesDepositChar)


2000
바뀜
2000


In [466]:
salesdepositInt = salesDepositChar.replace('억', '00000000')

In [467]:
salesdepositInt = int(salesdepositInt)

In [468]:
salesdepositInt

2000

In [469]:
try:
    salesmonthlyChar = salesForm[2]
except IndexError:
    pass

In [470]:
salesmonthlyChar

'55만원'

In [471]:
salesmonthlyInt=salesmonthlyChar.replace('만원', '0000')

In [472]:
salesmonthlyInt = int(salesmonthlyInt)

In [473]:
salesmonthlyInt

550000

In [591]:
if salesType=='전세':
    salesdepositInt = salesdepositInt + salesmonthlyInt
    salesDepositChar = salesDepositChar + salesmonthlyChar
    
print(salesdepositInt)
print(salesDepositChar)

2000
2000


In [357]:
unrefined_floor = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[1]/div')
total_floor = unrefined_floor[0].get_attribute('innerText')
total_floor = total_floor.split('/')
floor = total_floor[0]
floor

'2층 '

In [136]:
totalFloor = total_floor[1]
totalFloor =totalFloor.replace(' ', '')
totalFloor

'3층'

In [137]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')
area = unrefined_area[0].get_attribute('innerText')
area

'42.97 / 46.28㎡'

In [ ]:
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/button').click()

In [507]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')
supplyAreaChar = unrefined_area[0].get_attribute('innerText')
supplyAreaInt = supplyAreaChar.split('/')
supplyAreaInt = supplyAreaInt[1].replace('평', '')
supplyAreaInt = supplyAreaInt.strip()
supplyAreaInt = int(supplyAreaInt)

60
<class 'str'>
['10 ', ' 60평']
 60
60
<class 'int'>


60

In [549]:
unrefined_shortRent = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[5]/p')

In [550]:
shortRent = unrefined_shortRent[0].get_attribute('innerText')

In [551]:
shortRent

'가능'

In [552]:
# if shortRent=='불가능':
#     shortRent= False
# else:
#     shortRent= True

In [553]:
unrefined_management = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]')
unrefined_management = unrefined_management[0].get_attribute('innerText')
unrefined_management = unrefined_management.replace('\n', '')
unrefined_management

'5만 원(인터넷, 기타)'

In [554]:
unrefined_management = unrefined_management.replace(' ', '')
unrefined_management = unrefined_management.replace('(', ' ')
unrefined_management = unrefined_management.replace(')', ' ')
unrefined_management = unrefined_management.replace(',', ' ')
unrefined_management = unrefined_management.split(' ')

In [562]:
unrefined_management
# get_or_create()해야 할 듯 .

['인터넷', '기타', '']

In [556]:
managementPay = unrefined_management.pop(0)

In [557]:
managementPay

'5만원'

In [558]:
if managementPay.find('만원'):
    managementPay = managementPay.replace('만원', ' ')
    print(managementPay)
    managementPay = int(managementPay)
else: 
    managementPay= 0
print(managementPay)

5 
5


In [571]:
admin_list = []
admin_list = [item for item in unrefined_management if not item =='']

In [574]:
admin_list

['인터넷', '기타']

## 이게 다방이 방 내놓을때에는 관리비 포함 항목들을 오브젝트로 만들어놨는데 
## 필터링할때에는 포함 항목들이 해당되지 않는다.
## 이게 다방은 없는데 우리는 있는 기능으로 만들 수 있긴 있을 것 같다.

In [575]:
unrefined_parking = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p')

In [576]:
unrefined_parking[0].get_attribute('innerText')
# 관리비는 따로 필터링 하지 않아서 그냥 텍스트로만 저장

'3만 원'

In [578]:
try:
    unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/label')
    unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/span')
except NoSuchElementException:
    pass
# 한달 생활비가 있는 글이 있고 없는 글이 있다.

In [579]:
unrefined_living_expenses= unrefined_living_expenses[0].get_attribute('innerText')
living_expenses = unrefined_living_expenses.replace(' ', '')
living_expenses

'118만원+α'

In [580]:
living_expenses_detail = unrefined_living_expenses_detail[0].get_attribute('innerText')
living_expenses_detail

'(월세 + 관리비 + 주차비)'

In [581]:
unrefined_parking = driver.find_elements_by_xpath("/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p")
parking = unrefined_parking[0].get_attribute('innerText')
parking
# model에서 주차 비용 유무는 그냥 ios에서 보일때만 표시하고 django에 들어올때에는 없으면 안받고 있으면 받음.
# 주차비도 필터링에 조건 되는게 없으니까 따로 숫자 빼지는 않겠음.

'3만 원'

In [582]:
unrefined_moveIn = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[9]/div')
moveIn = unrefined_moveIn[0].get_attribute('innerText')
moveIn

'즉시 입주'

# 날짜도 데이터타임 필드인데 크롤링 시 문자형이 있어서 우선 이걸로 
# 이건 필드 두개 나눠서 크롤링 데이터는 문자형
# 생성하게 될 것들은 데이터타임필드로 하던 우선 보류

In [584]:
option_ins_list = []
try:
    unrefined_option = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div[1]/div')
except NoSuchElementException:
    pass
unrefined_option = unrefined_option[0].get_attribute('innerText')
unrefined_option = unrefined_option.split('\n\n')
option = unrefined_option
for ins in option:
    option_ins_list.append(ins)

In [585]:
option_ins_list

['에어컨', '세탁기', '침대', '책상', '옷장', '신발장', '냉장고', '인덕션', '전자레인지', '전자도어락']

In [76]:
unrefined_heatingType = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[3]/div')
heatingType = unrefined_heatingType[0].get_attribute('innerText')
heatingType

'개별난방'

In [77]:
unrefined_pet = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[6]/div')
pet = unrefined_pet[0].get_attribute('innerText')
pet

'불가능'

In [78]:
unfined_elevator = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[5]/div')
elevator = unfined_elevator[0].get_attribute('innerText')
elevator

'없음'

### 복층, 1.5룸, 주방 분리형 아직 못찾아서 안넣었음

In [79]:
unrefined_builtIn = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[4]/div')
builtIn = unrefined_builtIn[0].get_attribute('innerText')
builtIn

'아님'

In [80]:
unrefined_veranda = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[7]/div')
veranda = unrefined_veranda[0].get_attribute('innerText')
veranda

'없음'

In [586]:
unrefined_depositLoan = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[8]/div')
depositLoan = unrefined_depositLoan[0].get_attribute('innerText')
depositLoan

'불가능'

In [588]:
try:
    unrefined_securitySafety = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div[2]/div')
    security = unrefined_securitySafety[0].get_attribute('innerText')
        security= security.split('\n\n')
except IndexError:
    pass

In [590]:
security

NameError: name 'security' is not defined